# Recommender Systems

Let $Y \in R^{(m, k)}$ matrix of $m$ product ratings given by $k$ users. 
Assign $x^{(i)} \in R^n$ vector of features to each i-th product. 

By minimizing linear regression cost function, find $\theta^{(j)} \in R^n$ for each user so that 

$$y_{predicted}^{(i,j)} = (\theta^{(j)})^T x^{(i)}$$

gives the prediction of j-th user rating for i-th product.

In [6]:
# import libs
import sys
sys.path.append("../")

import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import re
from scipy import optimize
from collaborative_filtering import *

In [7]:
# load data
data = scipy.io.loadmat('../data/ex8_movies.mat')
y = data['Y']
r = data['R']

print("Y", y.shape)
print("R", r.shape)

Y (1682, 943)
R (1682, 943)


## Collaborative Filtering

Minimize linear regression cost function for both $X$ and $\Theta$:

$$J(x^{(0)}, \dots, x^{(i)}, \theta^{(0)}, \dots, \theta^{(j)}) = 
    \frac{1}{2} \sum_{(i,j):r(i,j)=1}{((\theta^{(j)})^T x^{(i)} - y^{(i,j)})^2} 
        + \frac{\lambda}{2} \sum_{j=0}^k (\theta^{(j)})^2
        + \frac{\lambda}{2} \sum_{i=0}^n (x^{(i)})^2$$
        
Where $r(i, j)=1$ when there is a rating given by j-th user to i-th product.

Gradients:

$$\frac{dJ}{d \theta^{(j)}_k} = \sum_{i:r(i,j)=1}{((\theta^{(j)})^T x^{(i)} - y^{(i,j)})} x^{(i)}_k
    + \lambda \theta^{(j)}_k$$
    
$$\frac{dJ}{d x^{(i)}_k} = \sum_{j:r(i,j)=1}{((\theta^{(j)})^T x^{(i)} - y^{(i,j)})} \theta^{(j)}_k
    + \lambda x^{(i)}_k$$

### Movie Rating Prediction

Use collaborative filtering algorithm to find recommended movies for a new user.

In [8]:
# load movie names database
def load_movies():
    movies = []
    p = re.compile("\s+(.*)\n")
    with open('../data/movie_ids.txt', encoding="iso-8859-1") as f:
        for line in f:
            movies.append(p.search(line).group(1))
            
    return movies


movies = load_movies()

In [9]:
# give ratings to some movies of the data set
my_ratings = np.zeros((y.shape[0], 1))  # m x 1
my_ratings[0] = 4  # Toy Story (1995)
my_ratings[97] = 2  # Silence of the Lambs (1991)
my_ratings[6] = 3
my_ratings[11] = 5
my_ratings[53] = 4
my_ratings[63] = 5
my_ratings[65] = 3
my_ratings[68] = 5
my_ratings[182] = 4
my_ratings[225] = 5
my_ratings[354] = 5

for m in np.argwhere(my_ratings > 0)[:,0]:
    print(my_ratings[m][0], movies[m])

4.0 Toy Story (1995)
3.0 Twelve Monkeys (1995)
5.0 Usual Suspects, The (1995)
4.0 Outbreak (1995)
5.0 Shawshank Redemption, The (1994)
3.0 While You Were Sleeping (1995)
5.0 Forrest Gump (1994)
2.0 Silence of the Lambs, The (1991)
4.0 Alien (1979)
5.0 Die Hard 2 (1990)
5.0 Sphere (1998)


In [11]:
# find new set of Theta and X for each user and movie

# add new ratings to Y and R matrices
y = np.hstack((y, my_ratings))
r = np.hstack((r, my_ratings > 0))

y_means, y_norm = mean_normalize_variables(y, r)

num_features = 10
theta = np.random.rand(y.shape[1], num_features)  # k x n
x = np.random.rand(y.shape[0], num_features)  # m x n

original_shapes, cx = unroll_list_of_matrices_to_vector([theta, x])
regularization_rate = 10

optimal_values = optimize.fmin_cg(regularized_cost_function,
                                  cx,
                                  fprime=regularized_cost_function_derivative,
                                  args=(y_norm, regularization_rate, original_shapes, r),
                                  maxiter=200)

new_theta, new_x = roll_vector_to_list_of_matrices(optimal_values, original_shapes)

         Current function value: 38979.628875
         Iterations: 200
         Function evaluations: 301
         Gradient evaluations: 301


Find top 20 recommendations.

In [12]:
# find recommendations for each movie
my_recommendations = new_x @ new_theta[-1,:].transpose().reshape((new_x.shape[1], 1)) + y_means
my_recommendations = np.hstack((
    np.arange(my_recommendations.shape[0], dtype=int).reshape(my_recommendations.shape), 
    my_recommendations
))

my_recommendations = my_recommendations[np.argsort(my_recommendations[:, 1])][::-1]

for index, rating in my_recommendations[0:20,:]:
    print(round(rating, 1), movies[int(index)])

5.0 Marlene Dietrich: Shadow and Light (1996) 
5.0 Someone Else's America (1995)
5.0 Great Day in Harlem, A (1994)
5.0 Aiqing wansui (1994)
5.0 Santa with Muscles (1996)
5.0 They Made Me a Criminal (1939)
5.0 Entertaining Angels: The Dorothy Day Story (1996)
5.0 Saint of Fort Washington, The (1993)
5.0 Star Kid (1997)
5.0 Prefontaine (1997)
4.6 Pather Panchali (1955)
4.5 Wrong Trousers, The (1993)
4.5 Shawshank Redemption, The (1994)
4.5 Maya Lin: A Strong Clear Vision (1994)
4.5 Star Wars (1977)
4.5 Anna (1996)
4.5 Some Mother's Son (1996)
4.5 Everest (1998)
4.5 Titanic (1997)
4.5 Wallace & Gromit: The Best of Aardman Animation (1996)
